In [29]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [30]:
TRAIN_TEXT_FILE_PATH = 'C:/Users/kvanc/session/document_classification/datasets/text.txt'

with open(TRAIN_TEXT_FILE_PATH, encoding='utf-8') as text_file:
    text_sample = text_file.readlines()
    

text_sample = ' '.join(text_sample)

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]

    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

In [31]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [32]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [33]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [34]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 500
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) % 50 == 0:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

Loss: 2.827471251487732
 обнеризни ито праризна на подно обноды вода порреробнасто обние подно прозние на подна на доберсе облато обно подна себет обнара обнарате подно обнарете на подна прода проды на постоды воды жизни на п
Loss: 1.2079283499717712
 орбитального морохного ультрафии с орбитального излучения на Марсе име существовала на Марсе ослаб, хотя необерса и сегодня на Марсе изменя на Марсе изменя на Марсе ослаб, хотя недетельные признакичес
Loss: 0.28650111734867095
 надежды все еще держатся, и многие думают, что жизнь, возможно, существовала на Марсе в прошлом.
 
 Поиски жизни на Марсе
 В последние годы орбитальный аппарат обнаружил метан в марсианская атмосфера 
Loss: 0.10993815556168557
 надежды для пессимистов жизни на других планетах.
 
 Во-первых, ученые пришли к выводу, что Марс, планета без жидкой воды, когда-то пережила близкий к глобальному потопу, все время отрицая, что такое 
Loss: 0.07372940815985203
 Марсе ослаб, хотя некоторые надежды все еще держатся, и многие д

In [35]:
model.eval()

print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.3, 
    prediction_len=1000, 
    start_text='. '
    )
)

. обнаружению признаков жизни. Два эксперимента не показали никаких признаков живых организмов, третий эксперимент имел слабые, но неоднозначные данные. Даже самые оптимистичные искатели внеземной жизни согласны с тем, что эти незначительные положительные признаки, вероятно, были результатом неорганических химических реакций в почве. Помимо жуткого холода и редкости воды, сегодня на Марсе есть и другие препятствия для жизни. Например, тонкая марсианская атмосфера не обеспечивает защиту солнечного ультрафиолетового излучения, которое летально для живых существ.
 
 С этими проблемами интерес к жизни на Марсе ослаб, хотя некоторые надежды все еще держатся, и многие думают, что жизнь, возможно, существовала на Марсе в прошлом. Возможно Красная планета когда-то имела гораздо более существенную атмосферу, чем сейчас, атмосферу, которая обеспечивала достаточное давление и тепло для поддержания жидкой воды.
 
 Исследовавшие поверхность Марса с 1976 году марсоходы, содержали три очень надежных 